In [23]:
#Setup Librearies
import os
import torch
import glob
import torchvision
import pathlib
import torch
import numpy as np
import torch.nn as nn
from PIL import Image
from torch import flatten
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable

In [2]:
#Check CUDA Usage
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [3]:
#Transformers
transformer = transforms.Compose([
  transforms.Resize((250,250)),
  transforms.RandomHorizontalFlip(),
  transforms.ToTensor(),
  transforms.Normalize([0.5,0.5,0.5],
                       [0.5,0.5,0.5])
])

In [4]:
#DataLoaders
train_path = './archive/train/'
validation_path = './archive/valid/'
train_loader = DataLoader(
    torchvision.datasets.ImageFolder(train_path, transform=transformer),
    batch_size=200, shuffle=True
)
validation_loader = DataLoader(
    torchvision.datasets.ImageFolder(validation_path, transform=transformer),
    batch_size=200, shuffle=True
)

In [5]:
#Categories
root = pathlib.Path(train_path)
classes = sorted([image.name.split('/')[-1] for image in root.iterdir()])

In [6]:
print(classes)

75


In [7]:
#CNN Network
class ConvNet(nn.Module):
    def __init__(self, num_classes=75):
        super(ConvNet, self).__init__()
        
        ## FEATURE 1
        #Inpupt Shape = (256, 3, 150, 150)
        self.conv1=nn.Conv2d(in_channels=3, out_channels=12, kernel_size=(5, 5))
        #Shape = (256, 12, 150, 150)
        self.bn1=nn.BatchNorm2d(num_features=12)
        #Shape = (256, 12, 150, 150)
        self.relu1=nn.ReLU()
        #Shape = (256, 12, 150, 150)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        #Shape = (256, 12, 75, 75)
        
        ## FEATURE 2
        self.conv2=nn.Conv2d(in_channels=12, out_channels=32, kernel_size=(5, 5))
        #Shape = (256, 20, 75, 75)
        self.bn2=nn.BatchNorm2d(num_features=32)
        #Shape = (256, 20, 75, 75)
        self.relu2=nn.ReLU()
        #Shape = (256, 20, 75, 75)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        ## FEATURE 3
        self.conv3=nn.Conv2d(in_channels=32, out_channels=50, kernel_size=(2, 2))
        #Shape = (256, 32, 75, 75)
        self.bn3=nn.BatchNorm2d(num_features=50)
        self.relu3=nn.ReLU()
        #Shape = (256, 32, 75, 75)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        ## FLATTEN
        self.fc1=nn.Linear(in_features=29*29*50, out_features=1000)
        self.relu4=nn.ReLU()
        
        self.fc2=nn.Linear(in_features=1000, out_features=num_classes)
        
    def forward(self, input):
        output = self.conv1 ( input )
        output = self.bn1 ( output )
        output = self.relu1 ( output )
        output = self.pool1 ( output )

        output = self.conv2 ( output )
        output = self.bn2 ( output )
        output = self.relu2 ( output )
        output = self.pool2 ( output )

        output = self.conv3 ( output )
        output = self.bn3 ( output )
        output = self.relu3 ( output )
        output = self.pool3 ( output )
        
        output = flatten(output, 1)
        output = self.fc1(output)
        output = self.relu4(output)
        
        output = self.fc2(output)

        return output
        

In [8]:
model = ConvNet().to(device)

In [9]:
#Optmizer and Loss Function
optimizer = Adam(model.parameters(), lr=0.001, weight_decay=0.0001)
loss_function = nn.CrossEntropyLoss()

In [10]:
num_epochs = 50

In [11]:
train_count=len(glob.glob(train_path+'/**/*.jpg'))
validation_count=len(glob.glob(validation_path+'/**/*.jpg'))

In [12]:
print(train_count, validation_count)

9285 375


In [13]:
#Model Training and saving best model
best_accuracy = 0.00
for epoch in range(num_epochs):
    #Evaluation and training on train dataset
    model.train()
    train_accuracy = 0.00
    train_loss = 0.00
    
    for i, (images, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        
        outputs = model(images)
        loss=loss_function(outputs,labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.cpu().data*images.size(0)
        _, predicition = torch.max(outputs.data, 1)
        
        train_accuracy += int(torch.sum(predicition==labels.data))
    
    train_accuracy=train_accuracy/train_count
    train_loss=train_loss/train_count
    
    #Evaluation on testing dataset
    model.eval()
    
    validation_accuracy = 0.00
    for i, (images, labels) in enumerate(validation_loader):
        optimizer.zero_grad()
        
        outputs = model(images)
        
        _, predicition = torch.max(outputs.data, 1)
        validation_accuracy += int(torch.sum(predicition==labels.data))
        
    validation_accuracy=validation_accuracy/validation_count
    
    print(f'Epoch: {epoch} Train Loss: {int(train_loss)} Train Accuracy: {train_accuracy} Validation Accuracy: {validation_accuracy}')
    
    #Save best model
    if validation_accuracy > best_accuracy:
        torch.save(model.state_dict(), 'best_checkpoint.model')
        best_accuracy = validation_accuracy
    

Epoch: 0 Train Loss: 12 Train Accuracy: 0.06225094238018309 Validation Accuracy: 0.12
Epoch: 1 Train Loss: 3 Train Accuracy: 0.17705977382875607 Validation Accuracy: 0.2693333333333333
Epoch: 2 Train Loss: 2 Train Accuracy: 0.3155627355950458 Validation Accuracy: 0.368
Epoch: 3 Train Loss: 2 Train Accuracy: 0.4355411954765751 Validation Accuracy: 0.44533333333333336
Epoch: 4 Train Loss: 1 Train Accuracy: 0.5224555735056543 Validation Accuracy: 0.536
Epoch: 5 Train Loss: 1 Train Accuracy: 0.5961227786752827 Validation Accuracy: 0.56
Epoch: 6 Train Loss: 1 Train Accuracy: 0.6693591814754981 Validation Accuracy: 0.6293333333333333
Epoch: 7 Train Loss: 0 Train Accuracy: 0.722778675282714 Validation Accuracy: 0.6666666666666666
Epoch: 8 Train Loss: 0 Train Accuracy: 0.7766289714593431 Validation Accuracy: 0.6346666666666667
Epoch: 9 Train Loss: 0 Train Accuracy: 0.8210016155088853 Validation Accuracy: 0.6426666666666667
Epoch: 10 Train Loss: 0 Train Accuracy: 0.8578352180936996 Validation A

In [ ]:
# Test NN Setup
checkpoint = torch.load('best_checkpoint.model')
model = ConvNet()
model.load_state_dict(checkpoint)
model.eval()


In [20]:
#Define Test Path
test_path = './archive/test/'

In [25]:
# Test Funtion
def test(img_path, transformer):
    image = Image.open(img_path)
    
    image_tensor = transformer(image).float()
    
    image_tensor = image_tensor.unsqueeze_(0)
    
    input = Variable(image_tensor)
    
    output = model(input)
    
    index = output.data.numpy().argmax()
    
    pred=classes[index]
    
    return pred

In [21]:
images_path=glob.glob(test_path+'/**/*.jpg')

In [26]:
# Test flux
pred_dict = dict()

for i in images_path:
    pred_dict[i[i[i.rfind('/')+1:].rfind('/')+1:]] = test(i, transformer)

pred_dict

{'./archive/test/CAIRNS BIRDWING/5.jpg': 'CAIRNS BIRDWING',
 './archive/test/CAIRNS BIRDWING/2.jpg': 'CAIRNS BIRDWING',
 './archive/test/CAIRNS BIRDWING/4.jpg': 'CAIRNS BIRDWING',
 './archive/test/CAIRNS BIRDWING/3.jpg': 'CAIRNS BIRDWING',
 './archive/test/CAIRNS BIRDWING/1.jpg': 'CAIRNS BIRDWING',
 './archive/test/PURPLISH COPPER/5.jpg': 'SILVER SPOT SKIPPER',
 './archive/test/PURPLISH COPPER/2.jpg': 'BROWN SIPROETA',
 './archive/test/PURPLISH COPPER/4.jpg': 'BLUE MORPHO',
 './archive/test/PURPLISH COPPER/3.jpg': 'SLEEPY ORANGE',
 './archive/test/PURPLISH COPPER/1.jpg': 'PURPLISH COPPER',
 './archive/test/CHESTNUT/5.jpg': 'CHESTNUT',
 './archive/test/CHESTNUT/2.jpg': 'CHESTNUT',
 './archive/test/CHESTNUT/4.jpg': 'COMMON BANDED AWL',
 './archive/test/CHESTNUT/3.jpg': 'CHESTNUT',
 './archive/test/CHESTNUT/1.jpg': 'CHESTNUT',
 './archive/test/EASTERN DAPPLE WHITE/5.jpg': 'EASTERN DAPPLE WHITE',
 './archive/test/EASTERN DAPPLE WHITE/2.jpg': 'LARGE MARBLE',
 './archive/test/EASTERN DAPPLE 

In [27]:
# Use the NN
use_path = './archive/6 images/'
use_images_path=glob.glob(use_path+'/*.jpg')

In [28]:
# Use flux
pred_dict = dict()

for i in use_images_path:
    pred_dict[i[i.rfind('/')+1:]] = test(i, transformer)

pred_dict

{'5.jpg': 'ADONIS',
 '2.jpg': 'ADONIS',
 '4.jpg': 'ADONIS',
 '3.jpg': 'ADONIS',
 '6.jpg': 'AN 88',
 '1.jpg': 'ADONIS',
 'Danaidae_Monarch_John_Jackman-M.jpg': 'MONARCH',
 'Painted Lady UPUR1_Ian H Leach.jpg': 'CHECQUERED SKIPPER'}